In [207]:
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
import pyspark
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf, concat
 
from pyspark import SQLContext
 
conf = pyspark.SparkConf() 
 
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sqlcontext = SQLContext(sc)

schema = StructType([
    StructField("ts", StringType(),True),    
    StructField("uid", StringType(),True),     
    StructField("origh", StringType(),True),         
    StructField("origp", StringType(),True),     
    StructField("resph", StringType(),True),      
    StructField("respp", StringType(),True),   
    StructField("proto", StringType(),True),     
    StructField("service" , StringType(),True),        
    StructField("duration", FloatType(),True),     
    StructField("origbytes", StringType(),True),     
    StructField("respbytes", StringType(),True),       
    StructField("connstate", StringType(),True),      
    StructField("localorig", StringType(),True),   
    StructField("missedbytes", StringType(),True),      
    StructField("history", StringType(),True),     
    StructField("origpkts", IntegerType(),True),     
    StructField("origipbytes", IntegerType(),True),       
    StructField("resppkts", IntegerType(),True),      
    StructField("respipbytes", IntegerType(),True),     
    StructField("tunnelparents", StringType(),True)    
              ])
        

df = sqlcontext.read.csv(path="/home/ubuntu/Documents/forensics/tail.log", sep="\t", schema=schema) 




In [208]:
colsInt = udf(lambda z: toInt(z), IntegerType())

spark.udf.register("colsInt", colsInt)

def toInt(s):
    if isinstance(s, str) == True:
        st = [str(ord(i)) for i in s]
        return(int(''.join(st)))
    else:
         return s


In [209]:
a = df.withColumn( 'iorigp',colsInt('origp'))
c = a.withColumn( 'irespp',colsInt('respp'))
d = c.withColumn( 'iproto',colsInt('proto'))
f = d.withColumn( 'iorigbytes',colsInt( 'origbytes'))
g = f.withColumn( 'irespbytes',colsInt('respbytes'))
h = g.withColumn(  'iorigpkts',colsInt( 'origpkts'))
i = h.withColumn( 'iorigipbytes',colsInt('origipbytes'))

columns =  ['iorigp','irespp','iproto','iorigbytes','irespbytes','iorigpkts','iorigipbytes']
    
    

In [210]:
vecAssembler = VectorAssembler(inputCols=columns, outputCol="features")
router = vecAssembler.transform(i)

In [211]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

 
kmeans = KMeans().setK(7).setSeed(1)
model = kmeans.fit(router)

In [212]:
 # Make predictions
predictions = model.transform(router)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)


 

 

Silhouette with squared euclidean distance = 0.6000000005495167
Cluster Centers: 
[4.95152000e+05 4.95151000e+05 2.00917452e+09 4.50000000e+01
 4.50000000e+01 1.00000000e+00 1.04000000e+02]
[1.15356816e+09 4.95748480e+07 1.17100112e+08 5.54856000e+05
 4.80000000e+01 6.00000000e+00 9.96000000e+02]
[4.95156000e+05 4.95156000e+05 1.17100112e+08 4.50000000e+01
 4.50000000e+01 2.00000000e+00 4.93000000e+02]
[4.95051000e+05 4.95051000e+05 1.17100112e+08 4.50000000e+01
 4.50000000e+01 1.00000000e+00 7.60000000e+01]
[1.06057775e+09 1.06057775e+09 1.17100112e+08 5.65648000e+05
 4.80000000e+01 2.00000000e+01 1.44000000e+03]
[5.10000000e+01 4.90000000e+01 2.00917452e+09 5.54848000e+05
 4.80000000e+01 2.50000000e+01 1.40000000e+03]
[5.10000000e+01 5.10000000e+01 2.00917452e+09 5.05152000e+05
 4.80000000e+01 3.00000000e+00 3.18000000e+02]


In [215]:
 
predictions.select("prediction").show()

+----------+
|prediction|
+----------+
|         1|
|         2|
|         3|
|         4|
|         4|
|         3|
|         5|
|         0|
|         6|
|         0|
+----------+

